<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/Email_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_hub as hub
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [ ]:
!python -m pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()
df = pd.read_csv("spam.csv", encoding='unicode_escape')

Saving spam.csv to spam.csv


In [ ]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
df_spam = df[df['Category']=='spam']

df_ham = df[df['Category']=='ham']

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [ ]:
df_balanced = pd.concat([df_spam , df_ham_downsampled])

In [ ]:
df_balanced['Category'].value_counts()


spam    747
ham     747
Name: Category, dtype: int64

In [ ]:
df_balanced.head()

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x:1 if x=='spam' else 0)

In [ ]:
df_balanced.sample(10)

,Category,Message,spam
518,spam,Your credits have been topped up for http://ww...,1
2390,ham,Boo. How's things? I'm back at home and a litt...,0
4652,spam,A £400 XMAS REWARD IS WAITING FOR YOU! Our com...,1
3196,ham,Great. P diddy is my neighbor and comes for to...,0
811,ham,S:)s.nervous &lt;#&gt; :),0
1332,ham,Good Morning plz call me sir,0
5104,ham,A Boy loved a gal. He propsd bt she didnt mind...,0
1521,spam,"URGENT! Your Mobile No was awarded a £2,000 Bo...",1
2473,spam,Final Chance! Claim ur £150 worth of discount ...,1
4140,ham,Ever green quote ever told by Jerry in cartoon...,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'],
                                                     stratify = df_balanced['spam'])

In [ ]:
!pip install tensorflow_text

In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4', trainable = False)

In [ ]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

In [ ]:
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])


In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Inputs (InputLayer)         [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['Inputs[0][0]']              
                             128),                                                                
                              'input_word_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                                

In [ ]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [ ]:
history = model.fit(X_train, y_train, epochs = 10)


Epoch 1/10
35/35 [==============================] - 32s 316ms/step - loss: 0.6188 - accuracy: 0.6732 - precision: 0.6803 - recall: 0.6536
Epoch 2/10
35/35 [==============================] - 11s 312ms/step - loss: 0.4997 - accuracy: 0.8259 - precision: 0.8098 - recall: 0.8518
Epoch 3/10
35/35 [==============================] - 11s 309ms/step - loss: 0.4424 - accuracy: 0.8509 - precision: 0.8382 - recall: 0.8696
Epoch 4/10
35/35 [==============================] - 11s 316ms/step - loss: 0.3922 - accuracy: 0.8759 - precision: 0.8611 - recall: 0.8964
Epoch 5/10
35/35 [==============================] - 11s 321ms/step - loss: 0.3661 - accuracy: 0.8830 - precision: 0.8783 - recall: 0.8893
Epoch 6/10
35/35 [==============================] - 11s 325ms/step - loss: 0.3428 - accuracy: 0.8982 - precision: 0.8832 - recall: 0.9179
Epoch 7/10
35/35 [==============================] - 12s 338ms/step - loss: 0.3257 - accuracy: 0.8875 - precision: 0.8741 - recall: 0.9054
Epoch 8/10
35/35 [================

In [ ]:
model.evaluate(X_test,y_test)



12/12 [==============================] - 5s 316ms/step - loss: 0.2790 - accuracy: 0.8930 - precision: 0.8387 - recall: 0.9733


[0.2789936363697052, 0.893048107624054, 0.8387096524238586, 0.9732620120048523]

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

12/12 [==============================] - 4s 324ms/step


In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
import seaborn as sns



In [ ]:
predict_text = [
                # Spam
                'We’d all like to get a $10,000 deposit on our bank accounts out of the blue, but winning a prize—especially if you’ve never entered a contest',
                'Netflix is sending you a refund of $12.99. Please reply with your bank account and routing number to verify and get your refund',
                'Your account is temporarily frozen. Please log in to to secure your account ',
                #ham
                'The article was published on 18th August itself',
                'Although we are unable to give you an exact time-frame at the moment, I would request you to stay tuned for any updates.',
                'The image you sent is a UI bug, I can check that your article is marked as regular and is not in the monetization program.'
]

In [ ]:
test_results = model.predict(predict_text)
output = np.where(test_results>0.5,'spam', 'ham')

1/1 [==============================] - 1s 765ms/step
